In [1]:
import QuantLib as ql

# Testing CDS-option value against cached values...

In [3]:
cachedToday = ql.Date(10,ql.December,2007)
ql.Settings.instance().evaluationDate = cachedToday

calendar = ql.TARGET()

riskFree = ql.RelinkableYieldTermStructureHandle()
riskFree.linkTo(ql.FlatForward(cachedToday,0.02,ql.Actual360()))

expiry = calendar.advance(cachedToday,9,ql.Months)
startDate = calendar.advance(expiry,1,ql.Months)
maturity = calendar.advance(startDate,7,ql.Years)

dayCounter = ql.Actual360()
convention = ql.ModifiedFollowing
notional = 1000000.0

In [5]:
hazardRate = ql.QuoteHandle(ql.SimpleQuote(0.001))
schedule = ql.Schedule(startDate,maturity, ql.Period(ql.Quarterly),
                       calendar, convention, convention,
                       ql.DateGeneration.Forward, False)

In [6]:
recoveryRate = 0.4
defaultProbability = ql.DefaultProbabilityTermStructureHandle(ql.FlatHazardRate(0, calendar, hazardRate, dayCounter))

In [7]:
swapEngine = ql.MidPointCdsEngine(defaultProbability, recoveryRate, riskFree)
swap = ql.CreditDefaultSwap(ql.Protection.Seller, notional, 0.001, schedule, convention, dayCounter)

In [8]:
swap.setPricingEngine(swapEngine)
strike = swap.fairSpread()

In [10]:
cdsVol = ql.QuoteHandle(ql.SimpleQuote(0.20))

In [11]:
underlying = ql.CreditDefaultSwap(ql.Protection.Seller, notional, strike, schedule,
                                 convention, dayCounter)
underlying.setPricingEngine(swapEngine)

In [12]:
exercise = ql.EuropeanExercise(expiry)

In [13]:
option1 = ql.CdsOption(underlying, exercise)

In [14]:
option1.setPricingEngine(ql.BlackCdsOptionEngine(defaultProbability, recoveryRate,
                                             riskFree, cdsVol))

In [15]:
cachedValue = 270.976348

In [16]:
option1.NPV()

270.97634753353395

In [17]:
underlying = ql.CreditDefaultSwap(ql.Protection.Buyer, notional, strike, schedule,
                                 convention, dayCounter)
underlying.setPricingEngine(swapEngine)

In [18]:
option2 = ql.CdsOption(underlying, exercise)
option2.setPricingEngine(ql.BlackCdsOptionEngine(defaultProbability, recoveryRate,
                                             riskFree, cdsVol))

In [19]:
option2.NPV()

270.9763475329989